In [1]:
import query.datasets.prelude
reload(query.datasets.prelude)
from query.datasets.prelude import *
from query.datasets.tvnews.shot_detect import shot_detect, shot_stitch
from query.datasets.tvnews.face_detect import face_detect
from query.datasets.tvnews.face_embed import face_embed
from query.datasets.tvnews.pose_detect import pose_detect
from query.datasets.tvnews.identity_detect import identity_detect
from query.datasets.tvnews.animatedness import shot_frame_to_detect

In [ ]:
dims = json.load(open('/app/dims.json'))
videos = list(tqdm(Video.objects.all()))
def foo(video):
    item = video.path.split('/')[-1].replace('.mp4', '')
    if item in dims:
        video.height = dims[item]
        video.save()
par_for(foo, videos)

In [2]:
def load_videos():
    log.debug('Fetching videos')
    return list(Video.objects.annotate(
        c=Subquery(
            Shot.objects.filter(video=OuterRef('pk')).values('video') \
            .annotate(c=Count('video')).values('c')
        )).filter(c__isnull=False).order_by('id'))

videos_with_shots = pcache.get('videos_with_shots', load_videos, method='pickle')
videos = videos_with_shots

In [3]:
with Timer('Detecting shots'):
    all_shots = shot_detect(videos)
    face_frame_per_shot = [[shot_frame_to_detect(shot) for shot in vid_shots]
                           for vid_shots in all_shots]

D 18-01-13 15:02:40 prelude.py:225] -- START: Detecting shots
D 18-01-13 15:03:58 prelude.py:232] -- END: Detecting shots -- 00:01:17


In [4]:
with Timer('Detecting sparse face'):
    all_faces = face_detect(videos, face_frame_per_shot)
    filtered_videos, filtered_frames, filtered_faces, filtered_shots = unzip(
        [(video, vid_frames, vid_faces, vid_shots)
         for video, vid_frames, vid_faces, vid_shots in zip(videos, face_frame_per_shot, all_faces, all_shots)
         if vid_faces is not None])

D 18-01-13 15:03:58 prelude.py:225] -- START: Detecting sparse face
D 18-01-13 15:04:47 prelude.py:232] -- END: Detecting sparse face -- 00:00:48


In [ ]:
#all_bboxes = pickle.load(open('/app/bboxes.pkl', 'rb'))

In [ ]:
#print(filtered_videos[2].path)

In [ ]:
#with Timer('Embedding faces'):

In [5]:
def output_name(video, frames):
    return video.path + '_embeddings_' + str(hash(tuple(frames)))

with make_scanner_db() as db:
    db._load_db_metadata()
    
filter2_videos, filter2_frames, filter2_faces, filter2_shots, embed_tables = unzip(
    [(video, frames, faces, shots, db.table(output_name(video, frames)))
          for (video, frames, faces, shots) in tqdm(zip(filtered_videos, filtered_frames, filtered_faces, filtered_shots))
          if db.has_table(output_name(video, frames)) and db.table(output_name(video, frames)).committed()])


100%|██████████| 30471/30471 [00:01<00:00, 22465.21it/s]


In [7]:
with Timer("Embedding faces"):
    def load_embs():
        log.debug('Loading embs')
        EMBEDDING_SIZE = 128
        def load((t, vid_faces)):
            embs = list(t.column('embeddings').load())
            arrays = [np.frombuffer(emb, dtype=np.float32) if emb is not None else [] for _, emb in embs]
            return [np.split(a, len(a) / 128) if len(a) > 0 else [] for a in arrays]
        return par_for(load, zip(embed_tables, filter2_faces), workers=8)

    #all_embs = pcache.get('all_embs', load_embs, method='pickle')

# from query.datasets.tvnews.embed_kernel import EmbedFaceKernel
# import query.datasets.tvnews.embed_kernel
# reload(query.datasets.tvnews.embed_kernel)

# from scannerpy.stdlib import writers
# import cv2
# img = cv2.imread("/app/test.jpg")
# bboxes = [
#     db.protobufs.BoundingBox(x1=0, y1=0, x2=img.shape[1] - 1, y2=img.shape[0] - 1)
# ]

# kernel = query.datasets.tvnews.embed_kernel.EmbedFaceKernel(None, db.protobufs)
# [emb] = kernel.execute(
#     [cv2.cvtColor(img, cv2.COLOR_RGB2BGR),
#      writers.bboxes([bboxes], db.protobufs)[0]])
# exemplar_vector = np.frombuffer(emb, dtype=np.float32)
# print(exemplar_vector.shape)



D 18-01-12 18:12:50 prelude.py:221] -- START: Embedding faces
D 18-01-12 18:31:44 prelude.py:228] -- END: Embedding faces -- 00:18:45


In [ ]:
import struct

serializers = {}
type_hashes = {}
def serializer(ty):
    def register_class(cls):
        serializers[ty] = cls
        type_hashes[hash(ty)] = ty
        return cls
    return register_class

def serialize(v):
    ty = type(v)
    if ty in serializers:
        serializer = serializers[ty]
        hsh = hash(ty)
    else:
        serializer = DefaultSerializer
        hsh = 0
    return '{}{}'.format(struct.pack('=i', hsh), serializer.serialize(v))

def _deserialize(s):
    hsh, s = struct.unpack('=i', s[:4])[0], s[4:]
    serializer = serializers[type_hashes[hsh]]
    return serializer.deserialize(s)

def deserialize(s):
    return _deserialize(s)[0]
    
class DefaultSerializer:
    @staticmethod
    def serialize(v):
        return serialize(pickle.dumps(v))
    
    @staticmethod
    def deserialize(s):
        ps, s = _deserialize(s)
        return pickle.loads(ps), s
    
type_hashes[0] = 0    
serializers[0] = DefaultSerializer    

@serializer(int)
class IntSerializer:
    @staticmethod
    def serialize(v):
        return struct.pack('=q', v)
    
    @staticmethod
    def deserialize(s):
        return struct.unpack('=q', s[:8])[0], s[8:]
    
@serializer(str)    
class StringSerializer:
    @staticmethod
    def serialize(v):
        return '{}{}'.format(serialize(len(v)), v)
    
    @staticmethod
    def deserialize(s):
        n, s = _deserialize(s)
        return s[:n], s[n:]

@serializer(np.array)    
class NumpySerializer:
    @staticmethod
    def serialize(v):
        dtype = serialize(pi)
        n = serialize(len(v))
        return '{}{}{}'.format(dtype, n, v.tobytes())
    
    @staticmethod
    def deserialize(s):
        dtype, s = _deserialize(s)
        n, s = _deserialize(s)
        return np.frombuffer(s[:n], dtype=dtype), s[n:]

@serializer(list)    
class ListSerializer:
    @staticmethod
    def serialize(v):
        n = serialize(len(v))
        return '{}{}'.format(
            n, ''.join([serialize(x) for x in v])
        )
    
    @staticmethod
    def deserialize(s):
        n, s = _deserialize(s)
        l = []
        for _ in range(n):
            x, s = _deserialize(s)
            l.append(x)
        return l, s

def test(x):
    assert(deserialize(serialize(x)) == x)

test(1)
test("hello")
test([1, "hello"])

# with Timer('myser'):
#     x = serialize(all_embs[:10])
# with Timer('mydeser'):
#     deserialize(x)
# with Timer('pickleser'):
#     x = pickle.dumps(all_embs[:10])
# with Timer('pickledeser'):
#     pickle.loads(x)

#print(len(pickle.dumps(all_embs[0])))
#print(sum([sum([e.nbytes for e in l]) for l in all_embs[0]]))


In [6]:
with Timer("Stitching shots"):
    import query.datasets.prelude
    reload(query.datasets.prelude)
    def load_stitches():
        log.debug('Computing stitches')
        import query.datasets.tvnews.shot_detect
        reload(query.datasets.tvnews.shot_detect)
        return query.datasets.tvnews.shot_detect.shot_stitch(filter2_videos, filter2_shots, filter2_frames, filter2_faces, all_embs)
    (stitched_shots, stitched_indices) = query.datasets.prelude.pcache.get('stitched_shots', load_stitches)

D 18-01-13 15:05:05 prelude.py:225] -- START: Stitching shots
D 18-01-13 15:05:58 prelude.py:232] -- END: Stitching shots -- 00:00:51


In [53]:
def gather(l, idx):
    return [l[i] for i in idx]

def gather2(l, idx):
    return [l[i][j] for i, j in idx]

(filter3_videos, filter3_frames, filter3_faces) = unzip([
    (video, gather(frames, idxs), gather(faces, idxs))
    for (video, frames, faces, idxs) in tqdm(zip(filter2_videos, filter2_frames, filter2_faces,  stitched_indices))
])

# (filter3_videos, filter3_frames, filter3_faces, filter3_embs) = unzip([
#     (video, gather(frames, idxs), gather(faces, idxs), gather(embs, idxs))
#     for (video, frames, faces, embs, idxs) in tqdm(zip(filter2_videos, filter2_frames, filter2_faces, all_embs, stitched_indices))
# ])



  0%|          | 0/30046 [00:00<?, ?it/s]

  2%|▏         | 532/30046 [00:00<00:05, 5253.22it/s]

  4%|▎         | 1116/30046 [00:00<00:05, 5544.02it/s]

  5%|▌         | 1599/30046 [00:00<00:05, 5304.94it/s]

  7%|▋         | 2155/30046 [00:00<00:05, 5307.88it/s]

  9%|▊         | 2617/30046 [00:00<00:06, 4483.29it/s]

 11%|█         | 3171/30046 [00:00<00:05, 4636.87it/s]

 13%|█▎        | 3780/30046 [00:00<00:05, 4784.45it/s]

 15%|█▍        | 4411/30046 [00:00<00:05, 4955.75it/s]

 17%|█▋        | 4984/30046 [00:00<00:04, 5030.35it/s]

 18%|█▊        | 5516/30046 [00:01<00:05, 4698.79it/s]

 20%|██        | 6113/30046 [00:01<00:05, 4781.62it/s]

 22%|██▏       | 6716/30046 [00:01<00:04, 4871.35it/s]

 24%|██▍       | 7297/30046 [00:01<00:04, 4934.61it/s]

 26%|██▋       | 7917/30046 [00:01<00:04, 5013.66it/s]

 28%|██▊       | 8482/30046 [00:01<00:04, 4775.41it/s]

 30%|███       | 9104/30046 [00:01<00:04, 4852.25it/s]

 32%|███▏      | 9671/30046 [00:01<00:04, 4893.01it/s]

 34%

In [10]:
show = Show.objects.get(name='The Rachel Maddow Show').id
indices = [i for i, video in enumerate(filter3_videos) if video.show_id == show]

with Timer('Detecting identities'):
    def load_identities():
        log.debug('Computing identities')
        import query.datasets.tvnews.identity_detect
        reload(query.datasets.tvnews.identity_detect)

        return query.datasets.tvnews.identity_detect.identity_detect(
           gather(filter3_videos, indices), "/app/rachel-maddow.jpg", gather(filter3_embs, indices))
    
    matching_indices = pcache.get('matching_indices', load_identities)
    
print(len(matching_indices), sum([len(l) for l in matching_indices]))

D 18-01-13 15:07:18 prelude.py:225] -- START: Detecting identities
D 18-01-13 15:07:18 prelude.py:232] -- END: Detecting identities -- 00:00:00


(876, 40341)


In [101]:
matching_indices_onelevel = [[j for j, k in l] for l in matching_indices]
matching_videos, matching_shots, matching_frames, matching_faces = \
    (gather(filter3_videos, indices),      
    [gather(shots, idxs) for (shots, idxs) in zip(gather(stitched_shots, indices), matching_indices_onelevel)],
    [gather(frames, idxs) for (frames, idxs) in zip(gather(filter3_frames, indices), matching_indices_onelevel)],
    [gather(faces, idxs) for (faces, idxs) in zip(gather(filter3_faces, indices), matching_indices_onelevel)])
    
#pose_blacklist = ['tvnews/videos/MSNBCW_20170708_010000_The_Rachel_Maddow_Show.mp4']    
pose_blacklist = ['tvnews/videos/MSNBCW_20170506_040000_The_Rachel_Maddow_Show.mp4']
    
matching2_videos, matching2_shots, matching2_frames, matching2_faces = unzip([
    (video, shots, frames, faces)
    for video, shots, frames, faces in zip(matching_videos, matching_shots, matching_frames, matching_faces)
    if len(shots) > 0 and video.path not in pose_blacklist
])
    
TARGET_FPS = 10
pose_frames = [
    sum([list(range(s['min_frame'], s['max_frame'], int(round(video.fps / TARGET_FPS)))) for s in shots], [])
    for (video, shots) in zip(matching2_videos, matching2_shots)    
]

In [95]:
import query.datasets.tvnews.pose_detect
reload(query.datasets.tvnews.pose_detect)
from query.datasets.tvnews.pose_detect import pose_detect

all_poses = pose_detect(matching2_videos, pose_frames)

100%|██████████| 829/829 [00:38<00:00, 21.47it/s]

(829, 879, 829)


In [99]:
import query.datasets.tvnews.pose_detect
reload(query.datasets.tvnews.pose_detect)
from query.datasets.tvnews.pose_detect import pose_track

pose_track(matching2_videos, matching2_frames, matching2_shots, matching2_faces, all_poses)

TypeError: 'int' object has no attribute '__getitem__'

In [ ]:
def bbox_montage((i, (video, frames, faces, matches))):
    if len(matches) == 0:
        print(video.path)
        return
    make_montage(
        video,
        [frames[j] for j, _ in matches],
        '/tmp/montage{}.jpg'.format(i),
        bboxes=[[faces[j][k]] for j, k in matches],
        progress=False)

In [70]:
import query.datasets.prelude
reload(query.datasets.prelude)
from query.datasets.prelude import *

# make_montage(filter3_videos[indices[i]], filter3_frames[indices[i]],
#              '/app/montage.jpg', filter3_faces[indices[i]], workers=96, progress=True)

def bbox_montage((i, (video, frames, faces, matches))):
    try:
        if len(matches) == 0:
            print(video.path)
            return
        bbox_map = defaultdict(list, {j: [faces[j][k]] for j, k in matches})
        make_montage(
            video,
            #[frames[j] for j, _ in matches],
            frames,
            '/tmp/montage{}.jpg'.format(i),
            bboxes=[bbox_map[i] for i in range(len(frames))],
            #bboxes=[[faces[j][k]] for j, k in matches],
            progress=False)
    except Exception:
        traceback.print_exc()
        print(video.path)
    
_ = par_for(bbox_montage,
        list(enumerate(zip(gather(filter3_videos, indices)[:100], gather(filter3_frames, indices), 
            gather(filter3_faces, indices), matching_indices))),
        process=True,
        workers=8)





  0%|          | 0/100 [00:00<?, ?it/s]



  1%|          | 1/100 [01:08<1:52:30, 68.18s/it]



  9%|▉         | 9/100 [01:24<14:18,  9.43s/it]  



 13%|█▎        | 13/100 [01:30<10:08,  6.99s/it]



 16%|█▌        | 16/100 [01:54<10:03,  7.18s/it]



 17%|█▋        | 17/100 [02:06<10:16,  7.43s/it]



 20%|██        | 20/100 [02:09<08:37,  6.47s/it]

tvnews/videos/MSNBCW_20150721_080000_The_Rachel_Maddow_Show.mp4






 21%|██        | 21/100 [02:14<08:26,  6.41s/it]



 24%|██▍       | 24/100 [02:38<08:22,  6.61s/it]



 25%|██▌       | 25/100 [02:41<08:04,  6.47s/it]



 27%|██▋       | 27/100 [02:44<07:25,  6.11s/it]



 28%|██▊       | 28/100 [02:50<07:17,  6.07s/it]



 32%|███▏      | 32/100 [03:03<06:29,  5.73s/it]



 33%|███▎      | 33/100 [03:15<06:36,  5.91s/it]



 35%|███▌      | 35/100 [03:24<06:20,  5.85s/it]



 36%|███▌      | 36/100 [03:27<06:09,  5.77s/it]



 38%|███▊      | 38/100 [03:46<06:08,  5.95s/it]



 40%|████      | 40/100 [03:52<05:48,  5.81s/it]



 41%|████      | 41/100 [04:05<05:52,  5.98s/it]



 42%|████▏     | 42/100 [04:06<05:40,  5.87s/it]



 43%|████▎     | 43/100 [04:22<05:47,  6.10s/it]



 45%|████▌     | 45/100 [04:25<05:24,  5.90s/it]

tvnews/videos/MSNBCW_20160901_013100_The_Rachel_Maddow_Show.mp4






 47%|████▋     | 47/100 [04:26<05:00,  5.67s/it]



 48%|████▊     | 48/100 [04:39<05:02,  5.82s/it]

tvnews/videos/MSNBCW_20170527_230000_The_Rachel_Maddow_Show.mp4
tvnews/videos/MSNBCW_20160813_010000_The_Rachel_Maddow_Show.mp4






 49%|████▉     | 49/100 [04:52<05:04,  5.97s/it]



 51%|█████     | 51/100 [05:30<05:17,  6.48s/it]



 60%|██████    | 60/100 [05:37<03:44,  5.62s/it]



 61%|██████    | 61/100 [05:47<03:41,  5.69s/it]



 63%|██████▎   | 63/100 [05:49<03:25,  5.54s/it]



 64%|██████▍   | 64/100 [05:49<03:16,  5.47s/it]



 65%|██████▌   | 65/100 [06:03<03:15,  5.59s/it]



 66%|██████▌   | 66/100 [06:04<03:07,  5.52s/it]



 67%|██████▋   | 67/100 [06:14<03:04,  5.59s/it]



 68%|██████▊   | 68/100 [06:21<02:59,  5.61s/it]



 70%|███████   | 70/100 [06:43<02:52,  5.76s/it]



 72%|███████▏  | 72/100 [06:45<02:37,  5.63s/it]



 73%|███████▎  | 73/100 [06:54<02:33,  5.67s/it]



 75%|███████▌  | 75/100 [07:06<02:22,  5.68s/it]



 76%|███████▌  | 76/100 [07:10<02:15,  5.67s/it]



 77%|███████▋  | 77/100 [07:14<02:09,  5.65s/it]



 79%|███████▉  | 79/100 [07:26<01:58,  5.65s/it]



 80%|████████  | 80/100 [07:43<01:55,  5.79s/it]



 83%|████████▎ | 83/100 [07:49<01:36,  5.66s/it]



 84%|███

In [ ]:
import requests
import cv2

par_for(make_montage,
        list(enumerate(zip(filter2_videos, [[s['min_frame'] for s in l] for l in stitched_shots][:100]))),
        process=True,
        workers=8)

In [ ]:
Shot.objects.bulk_create_copy({
    'min_frame': shot['min_frame'], 
    'max_frame': shot['max_frame'], 
    'labeler_id': shot['labeler'],
    'video_id': shot['video__id']
} for shot_list in tqdm(stitched_shots) for shot in shot_list])